# Exploratory Data Analysis

In [1]:
# this cell is for imports
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import pandas as pd
import collections
import os
import time
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/shunlin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# getting jokes from kaggle dataset
kaggle_jokes_raw = pd.read_csv('input/kaggle/shortjokes.csv')

In [3]:
# see the head of the jokes
kaggle_jokes_raw.head()

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [4]:
# build a set of vocab (characters)
joke_lst = kaggle_jokes_raw["Joke"].tolist()
text = joke_lst[0]
for joke in joke_lst[1:]:
    text += '\n\n'
    text += joke
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

98 unique characters


In [21]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [22]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\x08':   0,
  '\n':   1,
  '\x10':   2,
  ' ' :   3,
  '!' :   4,
  '"' :   5,
  '#' :   6,
  '$' :   7,
  '%' :   8,
  '&' :   9,
  "'" :  10,
  '(' :  11,
  ')' :  12,
  '*' :  13,
  '+' :  14,
  ',' :  15,
  '-' :  16,
  '.' :  17,
  '/' :  18,
  '0' :  19,
  ...
}


In [23]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'[me narrating' ---- characters mapped to int ---- > [62 80 72  3 81 68 85 85 68 87 76 81 74]


In [24]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

[
m
e
 
n


In [25]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTell'
'ing my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vam'
"pires... men.\n\nI've been going through a really rough period at work this week It's my own fault for "
'swapping my tampax for sand paper.\n\nIf I could have dinner with anyone, dead or alive... ...I would c'
"hoose alive. -B.J. Novak-\n\nTwo guys walk into a bar. The third guy ducks.\n\nWhy can't Barbie get pregn"


In [26]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [27]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTel'
Target data: 'me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTell'


In [28]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 62 ('[')
  expected output: 80 ('m')
Step    1
  input: 80 ('m')
  expected output: 72 ('e')
Step    2
  input: 72 ('e')
  expected output: 3 (' ')
Step    3
  input: 3 (' ')
  expected output: 81 ('n')
Step    4
  input: 81 ('n')
  expected output: 68 ('a')


In [29]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [30]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [31]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [32]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
        rnn(rnn_units,
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [33]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 98) # (batch_size, sequence_length, vocab_size)


In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           25088     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3935232   
_________________________________________________________________
dense_1 (Dense)              (64, None, 98)            100450    
Total params: 4,060,770
Trainable params: 4,060,770
Non-trainable params: 0
_________________________________________________________________


In [36]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [37]:
sampled_indices

array([55,  9, 22, 64, 84, 21, 87,  9, 35, 49, 74, 58, 38, 20, 55, 40, 13,
        8, 23, 83, 15, 67,  0, 34, 28, 36,  7, 23, 88, 38, 16, 21, 85, 40,
        3, 85, 85, 64, 11, 15,  9, 19, 69, 17, 38, 42, 38, 10, 45, 86, 70,
       88, 20, 18, 44, 65, 39, 67, 83, 58, 20,  4,  9, 79, 20, 72, 97, 90,
       81, 16, 59, 15, 97, 54, 50, 19,  1, 15, 12, 45, 85, 45, 22, 38, 90,
       41, 84, 95, 72, 18, 56, 16, 64, 30, 86, 83, 60, 79, 14,  8])

In [38]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ere both stuck up bitches.\n\nDid you hear the one about the perfume store? I heard their manufacturin'

Next Char Predictions: 
 "T&3]q2t&@NgWC1TE*%4p,`\x08?9A$4uC-2rE rr](,&0b.CGC'Jscu1/I^D`pW1!&l1e~wn-X,~SO0\n,)JrJ3CwFq|e/U-];spYl+%"


In [40]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 98)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.586542


In [41]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [42]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [43]:
EPOCHS=3

In [ ]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])


Epoch 1/3
  33/3440 [..............................] - ETA: 7:48:48 - loss: 3.9041

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"You are "))